# AC温度範囲分析 (A/C ON状態のみ)

このノートブックは、CleaとIsetanMitsukoshiの2つの建物のAC制御データを分析し、A/C ON状態のみのデータを使用して月別の温度統計を計算します。

## 主な機能
- 複数のCSVファイルの読み込みと統合
- A/C ON/OFF状態によるフィルタリング
- A/C Name別の月別温度分析（平均値、標準偏差）
- サンプル数分析とFan Speed頻度分析
- 建物別のExcelファイル生成

## 出力ファイル
- `AC_setvalue_range_analysis_Clea.xlsx`
- `AC_setvalue_range_analysis_IsetanMitsukoshi.xlsx`


In [136]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("ライブラリのインポート完了！")


ライブラリのインポート完了！


In [137]:
# データパス設定（正しいパスに修正）
data_path = "../../Clea/ac-control/"
print(f"データパス: {data_path}")

# 利用可能なファイルの確認
if os.path.exists(data_path):
    csv_files = glob.glob(os.path.join(data_path, "*.csv"))
    print(f"CSVファイル {len(csv_files)}個発見")
    
    # ファイル名サンプル出力
    for i, file in enumerate(csv_files[:5]):
        print(f"  {i+1}. {os.path.basename(file)}")
    
    if len(csv_files) > 5:
        print(f"  ... 他 {len(csv_files)-5}個のファイル")
else:
    print(f"パスが存在しません: {data_path}")
    print("現在の作業ディレクトリ:", os.getcwd())
    print("上位ディレクトリ確認:")
    print("  - ../../Clea/ 存在:", os.path.exists("../../Clea/"))
    print("  - ../../Clea/ac-control/ 存在:", os.path.exists("../../Clea/ac-control/"))


データパス: ../../Clea/ac-control/
CSVファイル 30個発見
  1. ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
  2. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
  3. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
  4. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
  5. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
  ... 他 25個のファイル


In [138]:
# データローディング関数
def load_csv_data(file_path):
    """CSVファイルをロードする関数"""
    try:
        df = pd.read_csv(file_path)
        print(f"ファイルロード成功: {os.path.basename(file_path)}")
        print(f"  - 行数: {len(df)}")
        print(f"  - 列数: {len(df.columns)}")
        print(f"  - 列名: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"ファイルロードエラー: {e}")
        return None

def list_csv_files(directory):
    """ディレクトリ内のすべてのCSVファイル一覧を返す"""
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    print(f"'{directory}' ディレクトリで {len(csv_files)}個のCSVファイル発見")
    return csv_files

print("データローディング関数定義完了！")


データローディング関数定義完了！


In [139]:
# 使用例（正しいパスに修正）
# 特定ファイルのロード
# df = load_csv_data("../../Clea/ac-control/ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv")

# またはファイル一覧から選択
# csv_files = list_csv_files("../../Clea/ac-control/")
# df = load_csv_data(csv_files[0])  # 最初のファイルをロード

print("データロード準備完了！")
print("上記のコメントを解除して、必要なデータをロードしてください。")
print("正しいパス: ../../Clea/ac-control/")


データロード準備完了！
上記のコメントを解除して、必要なデータをロードしてください。
正しいパス: ../../Clea/ac-control/


## すべてのCSVファイルの統合と月別分析

すべてのAC制御データを統合して月別温度分析を実行します。


In [140]:
# すべてのCSVファイルの統合と月別分析（2つの建物処理）
def load_all_csv_files(directory):
    """ディレクトリ内のすべてのCSVファイルをロードして統合"""
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    print(f"総 {len(csv_files)}個のCSVファイル発見")
    
    all_data = []
    
    for i, file_path in enumerate(csv_files):
        print(f"ローディング中... ({i+1}/{len(csv_files)}) {os.path.basename(file_path)}")
        
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"  エラー: {e}")
            continue
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\n統合完了！総 {len(combined_df)}個の行")
        return combined_df
    else:
        print("ロードされたデータがありません。")
        return None

def process_building_data(building_name, data_path):
    """特定の建物のデータを処理"""
    print(f"\n{'='*60}")
    print(f"=== {building_name} 建物データ処理 ===")
    print(f"{'='*60}")
    
    # すべてのデータをロード
    print(f"=== {building_name} CSVファイルロード ===")
    all_data = load_all_csv_files(data_path)
    
    if all_data is None:
        print(f"{building_name} データロード失敗")
        return None
    
    return all_data

# 2つの建物データ処理
buildings = {
    'Clea': '../../Clea/ac-control/',
    'IsetanMitsukoshi': '../../IsetanMitsukoshi/ac-control/'
}

building_data = {}

for building_name, data_path in buildings.items():
    print(f"\n{building_name} 建物データ処理開始...")
    data = process_building_data(building_name, data_path)
    if data is not None:
        building_data[building_name] = data
        print(f"{building_name} データ処理完了: {len(data)}個の行")
    else:
        print(f"{building_name} データ処理失敗")

print(f"\n総 {len(building_data)}個の建物データ処理完了")



Clea 建物データ処理開始...

=== Clea 建物データ処理 ===
=== Clea CSVファイルロード ===
総 30個のCSVファイル発見
ローディング中... (1/30) ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
ローディング中... (2/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
ローディング中... (3/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
ローディング中... (4/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
ローディング中... (5/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
ローディング中... (6/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-12-01-2024-12-31.csv
ローディング中... (7/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-01-01-2025-01-31.csv
ローディング中... (8/30) ac-control-71,70,72,73-logs-2024-11-01-2024-11-30.csv
ローディング中... (9

In [141]:
# データ構造確認と日付列処理
if all_data is not None:
    print("=== データ構造確認 ===")
    print(f"データ形状: {all_data.shape}")
    print(f"列一覧: {list(all_data.columns)}")
    
    # 日付/時間列の検索
    time_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    print(f"時間関連列: {time_columns}")
    
    # 温度関連列の検索
    temp_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['temp', '온도', 'temperature'])]
    print(f"温度関連列: {temp_columns}")
    
    # サンプルデータ確認
    print(f"\nサンプルデータ（最初の3行）:")
    print(all_data.head(3))
    
    # 各列のデータ型確認
    print(f"\nデータ型:")
    print(all_data.dtypes)


=== データ構造確認 ===
データ形状: (983682, 17)
列一覧: ['A/C Name', 'Datetime', 'Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'A/C ON/OFF', 'A/C Mode', 'A/C Fan Speed', 'Naive Energy Level', 'Airux Energy Level', 'Outdoor Room Temp.', 'Outdoor Set Temp.', 'Room Set Temp.', 'source_file', 'year_month', 'year', 'month']
時間関連列: ['Datetime']
温度関連列: ['Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'Outdoor Room Temp.', 'Outdoor Set Temp.', 'Room Set Temp.']

サンプルデータ（最初の3行）:
   A/C Name                         Datetime  Outdoor Temp.  Indoor Temp.  \
3      G-22 2025-04-30 23:55:00.084567+09:00            6.0          18.0   
12     G-21 2025-04-30 23:40:00.023055+09:00            6.0          19.0   
18     G-21 2025-04-30 23:35:00.072182+09:00            6.0          20.0   

    A/C Set Temperature A/C ON/OFF A/C Mode A/C Fan Speed  Naive Energy Level  \
3                  22.0         ON     HEAT        Medium                 100   
12                 26.0         ON     COOL        

In [142]:
# 日付列処理と月別分析（A/C ON状態のみフィルタリング）
def process_date_column(df, date_column):
    """日付列を処理して月情報を抽出"""
    if date_column not in df.columns:
        print(f"日付列 '{date_column}' が存在しません。")
        return df
    
    # 日付列をdatetimeに変換
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    
    # 月情報抽出
    df['year_month'] = df[date_column].dt.to_period('M')
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    
    return df

def analyze_monthly_temperature(df, indoor_temp_col, ac_set_temp_col):
    """月別温度分析（A/C ON状態のみ）"""
    if indoor_temp_col not in df.columns or ac_set_temp_col not in df.columns:
        print(f"必要な温度列が存在しません。")
        print(f"Indoor Temp: {indoor_temp_col in df.columns}")
        print(f"A/C Set Temperature: {ac_set_temp_col in df.columns}")
        return None
    
    # 月別グループ化と統計計算
    monthly_stats = df.groupby('year_month').agg({
        indoor_temp_col: ['mean', 'std', 'count'],
        ac_set_temp_col: ['mean', 'std', 'count']
    }).round(2)
    
    # 列名整理
    monthly_stats.columns = [
        'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
        'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
    ]
    
    # インデックスを文字列に変換
    monthly_stats.index = monthly_stats.index.astype(str)
    
    return monthly_stats

# 日付列処理とA/C ON状態フィルタリング
if all_data is not None:
    time_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    
    if time_columns:
        date_column = time_columns[0]
        print(f"日付列 '{date_column}' 使用")
        all_data = process_date_column(all_data, date_column)
        
        # A/C ON/OFF列検索とフィルタリング
        ac_onoff_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ["A/C ON/OFF", 'on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
        print(f"A/C ON/OFF関連列: {ac_onoff_columns}")
        
        if ac_onoff_columns:
            ac_onoff_col = ac_onoff_columns[0]
            print(f"A/C ON/OFF列 '{ac_onoff_col}' 使用")
            
            # フィルタリング前データ数
            total_rows = len(all_data)
            print(f"全体データ: {total_rows}個の行")
            
            # A/C ON状態のみフィルタリング
            ac_on_data = all_data[all_data[ac_onoff_col] == 'ON'].copy()
            ac_on_rows = len(ac_on_data)
            print(f"A/C ON状態データ: {ac_on_rows}個の行 ({ac_on_rows/total_rows*100:.1f}%)")
            
            if ac_on_rows > 0:
                # 月別分析（A/C ON状態のみ）
                indoor_temp_col = 'Indoor Temp.'
                ac_set_temp_col = 'A/C Set Temperature'
                
                monthly_analysis = analyze_monthly_temperature(ac_on_data, indoor_temp_col, ac_set_temp_col)
                
                if monthly_analysis is not None:
                    print("\n=== A/C ON状態月別温度分析結果 ===")
                    print(monthly_analysis)
                    
                    # フィルタリングされたデータをall_dataに更新
                    all_data = ac_on_data
                else:
                    print("A/C ON状態データで月別分析を実行できません。")
            else:
                print("A/C ON状態データがありません。")
        else:
            print("A/C ON/OFF列が見つかりません。全体データで分析を進行します。")
            
            # 月別分析（全体データ）
            indoor_temp_col = 'Indoor Temp.'
            ac_set_temp_col = 'A/C Set Temperature'
            
            monthly_analysis = analyze_monthly_temperature(all_data, indoor_temp_col, ac_set_temp_col)
            
            if monthly_analysis is not None:
                print("\n=== 月別温度分析結果 ===")
                print(monthly_analysis)
            else:
                print("月別分析を実行できません。")
    else:
        print("日付/時間列が見つかりません。")


日付列 'Datetime' 使用
A/C ON/OFF関連列: ['A/C ON/OFF']
A/C ON/OFF列 'A/C ON/OFF' 使用
全体データ: 983682個の行
A/C ON状態データ: 983682個の行 (100.0%)

=== A/C ON状態月別温度分析結果 ===
            Indoor_Temp_Mean  Indoor_Temp_Std  Indoor_Temp_Count  \
year_month                                                         
2024-07                27.43             1.89              84927   
2024-08                26.83             2.15              87083   
2024-09                25.69             2.00              83463   
2024-10                25.47             2.07              52638   
2024-11                23.88             2.80              41735   
2024-12                25.02             3.34              65915   
2025-01                24.56             3.77              55788   
2025-02                24.34             3.69              51607   
2025-03                25.77             2.80              56233   
2025-04                25.24             2.35              36186   
2025-05                25.44     

In [143]:
# 既存Excelファイル確認とA/C Name別分析（A/C ON状態のみ）
def check_existing_excel(filename):
    """既存Excelファイルのシート構造確認"""
    try:
        if os.path.exists(filename):
            # 既存ファイル読み込み
            excel_file = pd.ExcelFile(filename)
            print(f"既存Excelファイル発見: {filename}")
            print(f"既存シート一覧: {excel_file.sheet_names}")
            
            # 各シートの構造確認
            for sheet_name in excel_file.sheet_names:
                df = pd.read_excel(filename, sheet_name=sheet_name)
                print(f"\n=== {sheet_name} シート ===")
                print(f"行数: {len(df)}, 列数: {len(df.columns)}")
                print(f"列: {list(df.columns)}")
                if len(df) > 0:
                    print(f"サンプルデータ:")
                    print(df.head(3))
            return excel_file
        else:
            print(f"既存Excelファイルがありません: {filename}")
            return None
    except Exception as e:
        print(f"Excelファイル確認エラー: {e}")
        return None

def analyze_by_ac_name(df, indoor_temp_col, ac_set_temp_col):
    """A/C Name別で月別温度分析（A/C ON状態のみ）"""
    if 'A/C Name' not in df.columns:
        print("A/C Name列が存在しません。")
        return None
    
    # A/C Name別でグループ化
    ac_names = df['A/C Name'].unique()
    print(f"発見されたA/C Name: {ac_names}")
    
    results = {}
    
    for ac_name in ac_names:
        print(f"\n=== {ac_name} 分析中 ===")
        ac_data = df[df['A/C Name'] == ac_name].copy()
        
        if len(ac_data) == 0:
            continue
            
        # 月別グループ化と統計計算
        monthly_stats = ac_data.groupby('year_month').agg({
            indoor_temp_col: ['mean', 'std', 'count'],
            ac_set_temp_col: ['mean', 'std', 'count']
        }).round(2)
        
        # 列名整理
        monthly_stats.columns = [
            'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
            'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
        ]
        
        # インデックスを文字列に変換
        monthly_stats.index = monthly_stats.index.astype(str)
        
        results[ac_name] = monthly_stats
        print(f"  - 分析完了: {len(monthly_stats)}個の月")
    
    return results

# 既存Excelファイル確認
filename = "AC_setvalue_range_analysis.xlsx"
existing_excel = check_existing_excel(filename)


既存Excelファイル発見: AC_setvalue_range_analysis.xlsx
既存シート一覧: ['Indoortemp平均', '設定温度_平均値', 'Indoortemp標準偏差', '設定温度_標準偏差', '室内機別_サンプル数', 'FanSpeed頻度']

=== Indoortemp平均 シート ===
行数: 12, 列数: 27
列: ['Unnamed: 0', 'G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']
サンプルデータ:
  Unnamed: 0   G-22   G-21   G-24   G-23  D-1南1  D-3南2  D-6北1  D-5南1   A-26  \
0         1月  18.18  18.44  19.78    NaN  25.83  26.73  27.40  26.79  20.75   
1         2月  19.79  19.44  22.02    NaN  25.99  26.28  28.11  27.56  21.15   
2         3月  23.18  23.27  25.65  22.19  26.96  27.17  23.79  24.34  23.58   

   ...  D-4北2  E-15北3  D-7南2  D-8北2  E-16北4   A-25  E-12南4  D-2北1   F-19  \
0  ...  27.39   25.78  27.81  27.62   24.13  17.42   25.67   27.4  24.80   
1  ...    NaN   25.02    NaN    NaN   24.28  16.70   25.56    NaN  25.35   
2  ...  25.2

In [144]:
# A/C Name別分析実行とExcel更新（A/C ON状態のみ、既存フォーマットに合わせて）
if all_data is not None and 'A/C Name' in all_data.columns:
    print("\n=== A/C Name別月別分析実行（A/C ON状態のみ） ===")
    
    indoor_temp_col = 'Indoor Temp.'
    ac_set_temp_col = 'A/C Set Temperature'
    
    # A/C Name別分析
    ac_analysis_results = analyze_by_ac_name(all_data, indoor_temp_col, ac_set_temp_col)
    
    if ac_analysis_results:
        print(f"\n総 {len(ac_analysis_results)}個のA/C Nameに対する分析完了")
        
        # 各A/C Nameの分析結果プレビュー
        for ac_name, monthly_data in ac_analysis_results.items():
            print(f"\n--- {ac_name} 分析結果 ---")
            print(f"分析期間: {len(monthly_data)}個の月")
            print(f"月別平均 Indoor Temp: {monthly_data['Indoor_Temp_Mean'].mean():.2f}°C")
            print(f"月別平均 A/C Set Temp: {monthly_data['AC_Set_Temp_Mean'].mean():.2f}°C")
        
        # A/C ON状態サンプル数計算
        def calculate_ac_on_sample_counts(df):
            """A/C ON状態の月別サンプル数計算"""
            if 'A/C Name' not in df.columns:
                return None
            
            # A/C Name別月別サンプル数計算
            sample_counts = df.groupby(['A/C Name', 'year_month']).size().reset_index(name='sample_count')
            
            # 月別インデックス生成（1月, 2月, ..., 12月）
            months = [f"{i}月" for i in range(1, 13)]
            
            # A/C Name別データを月別に整理
            monthly_sample_data = {}
            
            for ac_name in df['A/C Name'].unique():
                monthly_values = {}
                
                # 該当A/C Nameの月別サンプル数
                ac_data = sample_counts[sample_counts['A/C Name'] == ac_name]
                
                for _, row in ac_data.iterrows():
                    # periodを月に変換（例: 2024-07 -> 7月）
                    month_period = row['year_month']
                    month_num = int(str(month_period).split('-')[1])
                    month_name = f"{month_num}月"
                    monthly_values[month_name] = row['sample_count']
                
                monthly_sample_data[ac_name] = monthly_values
            
            # DataFrame生成（月別が行、A/C Nameが列）
            df_samples = pd.DataFrame(monthly_sample_data, index=months)
            
            # 最初の列を月情報として追加
            df_samples.insert(0, 'Unnamed: 0', months)
            
            return df_samples
        
        # A/C Fan Speed頻度計算
        def calculate_fan_speed_frequency(df):
            """A/C Fan Speedの月別頻度計算（A/C ON状態のみ）"""
            if 'A/C Name' not in df.columns or 'A/C Fan Speed' not in df.columns:
                print("A/C NameまたはA/C Fan Speed列がありません。")
                return None
            
            # A/C ON/OFF列がある場合はON状態のみフィルタリング
            ac_onoff_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
            
            if ac_onoff_columns:
                ac_onoff_col = ac_onoff_columns[0]
                print(f"Fan Speed頻度計算: A/C ON/OFF列 '{ac_onoff_col}' 使用")
                
                # A/C ON状態のみフィルタリング
                ac_on_data = df[df[ac_onoff_col] == 'ON'].copy()
                print(f"Fan Speed頻度計算用データ: {len(ac_on_data)}個の行（A/C ON状態のみ）")
                
                if len(ac_on_data) == 0:
                    print("A/C ON状態データがないためFan Speed頻度を計算できません。")
                    return None
                
                # フィルタリングされたデータ使用
                df = ac_on_data
            else:
                print("A/C ON/OFF列が見つからないため全体データでFan Speed頻度を計算します。")
            
            # 月別インデックス生成（1月, 2月, ..., 12月）
            months = [f"{i}月" for i in range(1, 13)]
            
            # Fan Speed固有値確認
            fan_speeds = df['A/C Fan Speed'].unique()
            print(f"発見されたFan Speed値: {fan_speeds}")
            
            # Fan Speed別月別頻度計算
            fan_speed_frequency_data = []
            
            for month_name in months:
                month_num = int(month_name.replace('月', ''))
                
                for fan_speed in fan_speeds:
                    # 該当月の該当Fan Speed頻度計算
                    month_data = df[df['month'] == month_num]
                    fan_speed_count = len(month_data[month_data['A/C Fan Speed'] == fan_speed])
                    
                    fan_speed_frequency_data.append({
                        'Unnamed: 0': month_name,
                        'Unnamed: 1': fan_speed,
                        'frequency': fan_speed_count
                    })
            
            # DataFrame生成
            df_frequency = pd.DataFrame(fan_speed_frequency_data)
            
            # A/C Name別で列追加
            for ac_name in df['A/C Name'].unique():
                ac_frequency_values = []
                
                for month_name in months:
                    month_num = int(month_name.replace('月', ''))
                    
                    for fan_speed in fan_speeds:
                        # 該当A/C Nameの該当月、該当Fan Speed頻度
                        month_ac_data = df[(df['month'] == month_num) & (df['A/C Name'] == ac_name)]
                        fan_speed_count = len(month_ac_data[month_ac_data['A/C Fan Speed'] == fan_speed])
                        ac_frequency_values.append(fan_speed_count)
                
                df_frequency[ac_name] = ac_frequency_values
            
            print(f"Fan Speed頻度計算完了: {len(df_frequency)}個の行")
            return df_frequency
        
        # A/C ON状態サンプル数計算
        sample_counts_df = calculate_ac_on_sample_counts(all_data)
        
        # A/C Fan Speed頻度計算
        fan_speed_frequency_df = calculate_fan_speed_frequency(all_data)
        
        # Excelファイル更新（既存フォーマット維持）
        def update_excel_with_ac_data(excel_file, ac_analysis_results, sample_counts_df, fan_speed_frequency_df, filename):
            """既存ExcelファイルにA/C Name別データを既存フォーマットに合わせて入力（A/C ON状態のみ）"""
            try:
                # 既存ファイルがある場合は読み込み
                if excel_file is not None:
                    existing_sheets = {}
                    for sheet_name in excel_file.sheet_names:
                        existing_sheets[sheet_name] = pd.read_excel(filename, sheet_name=sheet_name)
                else:
                    existing_sheets = {}
                
                # 新しいExcelファイル生成
                with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                    # 既存シート維持しながら更新
                    for sheet_name, sheet_data in existing_sheets.items():
                        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
                    
                    # A/C Name別データを既存フォーマットに合わせて各シートに入力
                    if ac_analysis_results:
                        # 月別データを既存フォーマットに変換する関数
                        def create_monthly_dataframe(ac_analysis_results, data_type):
                            """A/C Name別分析結果を既存フォーマットのDataFrameに変換（A/C ON状態のみ）"""
                            # 月別インデックス生成（1月, 2月, ..., 12月）
                            months = [f"{i}月" for i in range(1, 13)]
                            
                            # A/C Name別データを月別に整理
                            monthly_data = {}
                            
                            for ac_name, monthly_stats in ac_analysis_results.items():
                                monthly_values = {}
                                
                                for month_period, row in monthly_stats.iterrows():
                                    # periodを月に変換（例: 2024-07 -> 7月）
                                    month_num = int(month_period.split('-')[1])
                                    month_name = f"{month_num}月"
                                    
                                    if data_type == 'indoor_mean':
                                        monthly_values[month_name] = row['Indoor_Temp_Mean']
                                    elif data_type == 'indoor_std':
                                        monthly_values[month_name] = row['Indoor_Temp_Std']
                                    elif data_type == 'ac_set_mean':
                                        monthly_values[month_name] = row['AC_Set_Temp_Mean']
                                    elif data_type == 'ac_set_std':
                                        monthly_values[month_name] = row['AC_Set_Temp_Std']
                                
                                monthly_data[ac_name] = monthly_values
                            
                            # DataFrame生成（月別が行、A/C Nameが列）
                            df = pd.DataFrame(monthly_data, index=months)
                            
                            # 最初の列を月情報として追加
                            df.insert(0, 'Unnamed: 0', months)
                            
                            return df
                        
                        # 1. Indoortemp平均シート（Indoor Temp月別平均 - A/C ON状態のみ）
                        indoor_avg_df = create_monthly_dataframe(ac_analysis_results, 'indoor_mean')
                        indoor_avg_df.to_excel(writer, sheet_name='Indoortemp平均', index=False)
                        
                        # 2. 設定温度_平均値シート（A/C Set Temperature月別平均 - A/C ON状態のみ）
                        ac_set_avg_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_mean')
                        ac_set_avg_df.to_excel(writer, sheet_name='設定温度_平均値', index=False)
                        
                        # 3. Indoortemp標準偏差シート（Indoor Temp月別標準偏差 - A/C ON状態のみ）
                        indoor_std_df = create_monthly_dataframe(ac_analysis_results, 'indoor_std')
                        indoor_std_df.to_excel(writer, sheet_name='Indoortemp標準偏差', index=False)
                        
                        # 4. 設定温度_標準偏差シート（A/C Set Temperature月別標準偏差 - A/C ON状態のみ）
                        ac_set_std_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_std')
                        ac_set_std_df.to_excel(writer, sheet_name='設定温度_標準偏差', index=False)
                    
                    # 5. 室内機別_サンプル数シート（A/C ON状態サンプル数）
                    if sample_counts_df is not None:
                        sample_counts_df.to_excel(writer, sheet_name='室内機別_サンプル数', index=False)
                        print("室内機別_サンプル数シートにA/C ON状態サンプル数入力完了")
                    
                    # 6. FanSpeed頻度シート（A/C Fan Speed頻度）
                    if fan_speed_frequency_df is not None:
                        fan_speed_frequency_df.to_excel(writer, sheet_name='FanSpeed頻度', index=False)
                        print("FanSpeed頻度シートにA/C Fan Speed頻度入力完了")
                
                print(f"Excelファイル更新完了: {filename}")
                return True
                
            except Exception as e:
                print(f"Excelファイル更新エラー: {e}")
                return False
        
        # Excelファイル更新実行
        print("\n=== Excelファイル更新実行（A/C ON状態のみ） ===")
        success = update_excel_with_ac_data(existing_excel, ac_analysis_results, sample_counts_df, fan_speed_frequency_df, filename)
        
        if success:
            print(f"\n=== 更新されたシート一覧（A/C ON状態のみ） ===")
            print("1. Indoortemp平均 - Indoor Temp月別平均（A/C ON状態のみ）")
            print("2. 設定温度_平均値 - A/C Set Temperature月別平均（A/C ON状態のみ）") 
            print("3. Indoortemp標準偏差 - Indoor Temp月別標準偏差（A/C ON状態のみ）")
            print("4. 設定温度_標準偏差 - A/C Set Temperature月別標準偏差（A/C ON状態のみ）")
            print("5. 室内機別_サンプル数 - A/C ON状態月別サンプル数")
            print("6. FanSpeed頻度 - A/C Fan Speed月別頻度")
            
            # 更新されたファイル確認
            updated_excel = pd.ExcelFile(filename)
            print(f"\n最終シート一覧: {updated_excel.sheet_names}")
        else:
            print("Excelファイル更新に失敗しました。")
    else:
        print("A/C Name別分析を実行できません。")
else:
    print("A/C Name列がないかデータがありません。")
    if all_data is not None:
        print(f"利用可能な列: {list(all_data.columns)}")



=== A/C Name別月別分析実行（A/C ON状態のみ） ===
発見されたA/C Name: ['G-22' 'G-21' 'G-24' 'G-23' 'D-1南1' 'D-3南2' 'D-6北1' 'D-5南1' 'A-26' 'E-17'
 'E-11南3' 'E-9南1' 'E-14北2' 'E-13北1' 'E-10南2' 'F-18' 'D-4北2' 'E-15北3'
 'D-7南2' 'D-8北2' 'E-16北4' 'A-25' 'E-12南4' 'D-2北1' 'F-19' 'F-20']

=== G-22 分析中 ===
  - 分析完了: 15個の月

=== G-21 分析中 ===
  - 分析完了: 15個の月

=== G-24 分析中 ===
  - 分析完了: 15個の月

=== G-23 分析中 ===
  - 分析完了: 7個の月

=== D-1南1 分析中 ===
  - 分析完了: 15個の月

=== D-3南2 分析中 ===
  - 分析完了: 15個の月

=== D-6北1 分析中 ===
  - 分析完了: 15個の月

=== D-5南1 分析中 ===
  - 分析完了: 15個の月

=== A-26 分析中 ===
  - 分析完了: 15個の月

=== E-17 分析中 ===
  - 分析完了: 15個の月

=== E-11南3 分析中 ===
  - 分析完了: 15個の月

=== E-9南1 分析中 ===
  - 分析完了: 15個の月

=== E-14北2 分析中 ===
  - 分析完了: 15個の月

=== E-13北1 分析中 ===
  - 分析完了: 15個の月

=== E-10南2 分析中 ===
  - 分析完了: 15個の月

=== F-18 分析中 ===
  - 分析完了: 15個の月

=== D-4北2 分析中 ===
  - 分析完了: 14個の月

=== E-15北3 分析中 ===
  - 分析完了: 15個の月

=== D-7南2 分析中 ===
  - 分析完了: 14個の月

=== D-8北2 分析中 ===
  - 分析完了: 13個の月

=== E-16北4 分析中 ===
  - 分析完了: 15個の月

=== A

In [145]:
# 既存Excelファイルのシート構造詳細確認（FanSpeed頻度シート含む）
filename = "AC_setvalue_range_analysis.xlsx"

if os.path.exists(filename):
    print(f"=== {filename} ファイル構造分析 ===")
    
    # Excelファイル読み込み
    excel_file = pd.ExcelFile(filename)
    print(f"シート一覧: {excel_file.sheet_names}")
    
    # FanSpeed頻度シートがあるか確認
    fanspeed_sheet_name = None
    for sheet_name in excel_file.sheet_names:
        if 'FanSpeed' in sheet_name or '頻度' in sheet_name:
            fanspeed_sheet_name = sheet_name
            break
    
    if fanspeed_sheet_name:
        print(f"\n{'='*60}")
        print(f"発見されたFanSpeed頻度シート: {fanspeed_sheet_name}")
        print(f"{'='*60}")
        
        df = pd.read_excel(filename, sheet_name=fanspeed_sheet_name)
        print(f"行数: {len(df)}, 列数: {len(df.columns)}")
        print(f"インデックスタイプ: {type(df.index)}")
        print(f"インデックス値: {list(df.index)}")
        print(f"列名: {list(df.columns)}")
        
        if len(df) > 0:
            print(f"\n全体データ:")
            print(df)
            
            # 各列のデータ型確認
            print(f"\nデータ型:")
            print(df.dtypes)
            
            # NaN値確認
            print(f"\nNaN値個数:")
            print(df.isnull().sum())
            
            # 最初の列の固有値確認（月情報）
            if 'Unnamed: 0' in df.columns:
                print(f"\n最初の列（月）固有値:")
                print(df['Unnamed: 0'].unique())
            
            # 2番目の列の固有値確認（Fan Speed情報）
            if 'Unnamed: 1' in df.columns:
                print(f"\n2番目の列（Fan Speed）固有値:")
                print(df['Unnamed: 1'].unique())
        else:
            print("データが空です。")
    else:
        print("FanSpeed頻度関連シートが見つかりません。")
    
    # 他のシートも簡単に確認
    for sheet_name in excel_file.sheet_names:
        if sheet_name != fanspeed_sheet_name:
            print(f"\n{'='*30}")
            print(f"シート名: {sheet_name}")
            print(f"{'='*30}")
            
            df = pd.read_excel(filename, sheet_name=sheet_name)
            print(f"行数: {len(df)}, 列数: {len(df.columns)}")
            print(f"列名: {list(df.columns)}")
            
            if len(df) > 0:
                print(f"\nサンプルデータ:")
                print(df.head(3))
else:
    print(f"ファイルが存在しません: {filename}")


=== AC_setvalue_range_analysis.xlsx ファイル構造分析 ===
シート一覧: ['Indoortemp平均', '設定温度_平均値', 'Indoortemp標準偏差', '設定温度_標準偏差', '室内機別_サンプル数', 'FanSpeed頻度']

発見されたFanSpeed頻度シート: FanSpeed頻度
行数: 60, 列数: 29
インデックスタイプ: <class 'pandas.core.indexes.range.RangeIndex'>
インデックス値: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
列名: ['Unnamed: 0', 'Unnamed: 1', 'frequency', 'G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']

全体データ:
   Unnamed: 0 Unnamed: 1  frequency  G-22  G-21  G-24  G-23  D-1南1  D-3南2  \
0          1月     Medium      19630  1286  1626    50     0   6642   6642   
1          1月        Low      35902     0     0     0     0   2206   220

In [146]:
# 2つの建物データ処理と分析
def process_building_analysis(building_name, data_path):
    """特定の建物のデータをロードして分析"""
    print(f"\n{'='*60}")
    print(f"=== {building_name} 建物データ処理 ===")
    print(f"{'='*60}")
    
    # データロード
    csv_files = glob.glob(os.path.join(data_path, "*.csv"))
    print(f"{building_name}: 総 {len(csv_files)}個のCSVファイル発見")
    
    all_data = []
    for i, file_path in enumerate(csv_files):
        print(f"ローディング中... ({i+1}/{len(csv_files)}) {os.path.basename(file_path)}")
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"  エラー: {e}")
            continue
    
    if not all_data:
        print(f"{building_name} データロード失敗")
        return None
    
    # データ統合
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"{building_name} 統合完了: {len(combined_df)}個の行")
    
    # 日付列処理
    time_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    if time_columns:
        date_column = time_columns[0]
        print(f"{building_name} 日付列 '{date_column}' 使用")
        combined_df[date_column] = pd.to_datetime(combined_df[date_column], errors='coerce')
        combined_df['year_month'] = combined_df[date_column].dt.to_period('M')
        combined_df['year'] = combined_df[date_column].dt.year
        combined_df['month'] = combined_df[date_column].dt.month
    
    # A/C ON/OFFフィルタリング
    ac_onoff_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
    if ac_onoff_columns:
        ac_onoff_col = ac_onoff_columns[0]
        print(f"{building_name} A/C ON/OFF列 '{ac_onoff_col}' 使用")
        
        total_rows = len(combined_df)
        ac_on_data = combined_df[combined_df[ac_onoff_col] == 'ON'].copy()
        ac_on_rows = len(ac_on_data)
        print(f"{building_name} A/C ON状態データ: {ac_on_rows}個の行 ({ac_on_rows/total_rows*100:.1f}%)")
        
        if ac_on_rows > 0:
            return ac_on_data
        else:
            print(f"{building_name} A/C ON状態データがありません。")
            return None
    else:
        print(f"{building_name} A/C ON/OFF列が見つかりません。")
        return combined_df

# 2つの建物データ処理
buildings = {
    'Clea': '../../Clea/ac-control/',
    'IsetanMitsukoshi': '../../IsetanMitsukoshi/ac-control/'
}

building_data = {}

for building_name, data_path in buildings.items():
    print(f"\n{building_name} 建物データ処理開始...")
    data = process_building_analysis(building_name, data_path)
    if data is not None:
        building_data[building_name] = data
        print(f"{building_name} データ処理完了: {len(data)}個の行")
    else:
        print(f"{building_name} データ処理失敗")

print(f"\n総 {len(building_data)}個の建物データ処理完了")



Clea 建物データ処理開始...

=== Clea 建物データ処理 ===
Clea: 総 30個のCSVファイル発見
ローディング中... (1/30) ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
ローディング中... (2/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
ローディング中... (3/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
ローディング中... (4/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
ローディング中... (5/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
ローディング中... (6/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-12-01-2024-12-31.csv
ローディング中... (7/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-01-01-2025-01-31.csv
ローディング中... (8/30) ac-control-71,70,72,73-logs-2024-11-01-2024-11-30.csv
ローディング中... (9/30) ac-control-50

In [147]:
# 各建物別分析とExcelファイル生成
def analyze_by_ac_name(df, indoor_temp_col, ac_set_temp_col):
    """A/C Name別月別温度分析（A/C ON状態のみ）"""
    if 'A/C Name' not in df.columns:
        print("A/C Name列がありません。")
        return None
    
    ac_names = df['A/C Name'].unique()
    print(f"総 {len(ac_names)}個のA/C Name発見")
    
    ac_analysis_results = {}
    
    for ac_name in ac_names:
        print(f"\n=== {ac_name} 分析中 ===")
        
        # 該当A/C Nameのデータのみフィルタリング
        ac_data = df[df['A/C Name'] == ac_name].copy()
        
        if len(ac_data) == 0:
            print(f"  {ac_name} データがありません。")
            continue
        
        # 月別グループ化と統計計算
        monthly_stats = ac_data.groupby('year_month').agg({
            indoor_temp_col: ['mean', 'std', 'count'],
            ac_set_temp_col: ['mean', 'std', 'count']
        }).round(2)
        
        # 列名整理
        monthly_stats.columns = [
            'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
            'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
        ]
        
        # インデックスを文字列に変換
        monthly_stats.index = monthly_stats.index.astype(str)
        
        ac_analysis_results[ac_name] = monthly_stats
        print(f"  分析完了: {len(monthly_stats)}個の月")
    
    return ac_analysis_results

def calculate_ac_on_sample_counts(df):
    """A/C ON状態の月別サンプル数計算"""
    if 'A/C Name' not in df.columns:
        return None
    
    # A/C Name別月別サンプル数計算
    sample_counts = df.groupby(['A/C Name', 'year_month']).size().reset_index(name='sample_count')
    
    # 月別インデックス生成（1月, 2月, ..., 12月）
    months = [f"{i}月" for i in range(1, 13)]
    
    # A/C Name別データを月別に整理
    monthly_sample_data = {}
    
    for ac_name in df['A/C Name'].unique():
        monthly_values = {}
        
        # 該当A/C Nameの月別サンプル数
        ac_data = sample_counts[sample_counts['A/C Name'] == ac_name]
        
        for _, row in ac_data.iterrows():
            # periodを月に変換（例: 2024-07 -> 7月）
            month_period = row['year_month']
            month_num = int(str(month_period).split('-')[1])
            month_name = f"{month_num}月"
            monthly_values[month_name] = row['sample_count']
        
        monthly_sample_data[ac_name] = monthly_values
    
    # DataFrame生成（月別が行、A/C Nameが列）
    df_samples = pd.DataFrame(monthly_sample_data, index=months)
    
    # 最初の列を月情報として追加
    df_samples.insert(0, 'Unnamed: 0', months)
    
    return df_samples

def calculate_fan_speed_frequency(df):
    """A/C Fan Speedの月別頻度計算（A/C ON状態のみ）"""
    if 'A/C Name' not in df.columns or 'A/C Fan Speed' not in df.columns:
        print("A/C NameまたはA/C Fan Speed列がありません。")
        return None
    
    # 月別インデックス生成（1月, 2月, ..., 12月）
    months = [f"{i}月" for i in range(1, 13)]
    
    # Fan Speed固有値確認
    fan_speeds = df['A/C Fan Speed'].unique()
    print(f"発見されたFan Speed値: {fan_speeds}")
    
    # Fan Speed別月別頻度計算
    fan_speed_frequency_data = []
    
    for month_name in months:
        month_num = int(month_name.replace('月', ''))
        
        for fan_speed in fan_speeds:
            # 該当月の該当Fan Speed頻度計算
            month_data = df[df['month'] == month_num]
            fan_speed_count = len(month_data[month_data['A/C Fan Speed'] == fan_speed])
            
            fan_speed_frequency_data.append({
                'Unnamed: 0': month_name,
                'Unnamed: 1': fan_speed,
                'frequency': fan_speed_count
            })
    
    # DataFrame生成
    df_frequency = pd.DataFrame(fan_speed_frequency_data)
    
    # A/C Name別で列追加
    for ac_name in df['A/C Name'].unique():
        ac_frequency_values = []
        
        for month_name in months:
            month_num = int(month_name.replace('月', ''))
            
            for fan_speed in fan_speeds:
                # 該当A/C Nameの該当月、該当Fan Speed頻度
                month_ac_data = df[(df['month'] == month_num) & (df['A/C Name'] == ac_name)]
                fan_speed_count = len(month_ac_data[month_ac_data['A/C Fan Speed'] == fan_speed])
                ac_frequency_values.append(fan_speed_count)
        
        df_frequency[ac_name] = ac_frequency_values
    
    print(f"Fan Speed頻度計算完了: {len(df_frequency)}個の行")
    return df_frequency

def create_monthly_dataframe(ac_analysis_results, data_type):
    """A/C Name別分析結果を既存フォーマットのDataFrameに変換（A/C ON状態のみ）"""
    # 月別インデックス生成（1月, 2月, ..., 12月）
    months = [f"{i}月" for i in range(1, 13)]
    
    # A/C Name別データを月別に整理
    monthly_data = {}
    
    for ac_name, monthly_stats in ac_analysis_results.items():
        monthly_values = {}
        
        for month_period, row in monthly_stats.iterrows():
            # periodを月に変換（例: 2024-07 -> 7月）
            month_num = int(month_period.split('-')[1])
            month_name = f"{month_num}月"
            
            if data_type == 'indoor_mean':
                monthly_values[month_name] = row['Indoor_Temp_Mean']
            elif data_type == 'indoor_std':
                monthly_values[month_name] = row['Indoor_Temp_Std']
            elif data_type == 'ac_set_mean':
                monthly_values[month_name] = row['AC_Set_Temp_Mean']
            elif data_type == 'ac_set_std':
                monthly_values[month_name] = row['AC_Set_Temp_Std']
        
        monthly_data[ac_name] = monthly_values
    
    # DataFrame生成（月別が行、A/C Nameが列）
    df = pd.DataFrame(monthly_data, index=months)
    
    # 最初の列を月情報として追加
    df.insert(0, 'Unnamed: 0', months)
    
    return df

def update_excel_with_building_data(building_name, ac_analysis_results, sample_counts_df, fan_speed_frequency_df):
    """建物別Excelファイル生成"""
    filename = f"AC_setvalue_range_analysis_{building_name}.xlsx"
    
    try:
        # 新しいExcelファイル生成
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # A/C Name別データを既存フォーマットに合わせて各シートに入力
            if ac_analysis_results:
                # 1. Indoortemp平均シート（Indoor Temp月別平均 - A/C ON状態のみ）
                indoor_avg_df = create_monthly_dataframe(ac_analysis_results, 'indoor_mean')
                indoor_avg_df.to_excel(writer, sheet_name='Indoortemp平均', index=False)
                
                # 2. 設定温度_平均値シート（A/C Set Temperature月別平均 - A/C ON状態のみ）
                ac_set_avg_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_mean')
                ac_set_avg_df.to_excel(writer, sheet_name='設定温度_平均値', index=False)
                
                # 3. Indoortemp標準偏差シート（Indoor Temp月別標準偏差 - A/C ON状態のみ）
                indoor_std_df = create_monthly_dataframe(ac_analysis_results, 'indoor_std')
                indoor_std_df.to_excel(writer, sheet_name='Indoortemp標準偏差', index=False)
                
                # 4. 設定温度_標準偏差シート（A/C Set Temperature月別標準偏差 - A/C ON状態のみ）
                ac_set_std_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_std')
                ac_set_std_df.to_excel(writer, sheet_name='設定温度_標準偏差', index=False)
            
            # 5. 室内機別_サンプル数シート（A/C ON状態サンプル数）
            if sample_counts_df is not None:
                sample_counts_df.to_excel(writer, sheet_name='室内機別_サンプル数', index=False)
                print(f"{building_name}: 室内機別_サンプル数シートにA/C ON状態サンプル数入力完了")
            
            # 6. FanSpeed頻度シート（A/C Fan Speed頻度）
            if fan_speed_frequency_df is not None:
                fan_speed_frequency_df.to_excel(writer, sheet_name='FanSpeed頻度', index=False)
                print(f"{building_name}: FanSpeed頻度シートにA/C Fan Speed頻度入力完了")
        
        print(f"{building_name}: Excelファイル生成完了 - {filename}")
        return True
        
    except Exception as e:
        print(f"{building_name}: Excelファイル生成エラー - {e}")
        return False

# 各建物別分析とExcelファイル生成
for building_name, data in building_data.items():
    print(f"\n{'='*60}")
    print(f"=== {building_name} 建物分析とExcelファイル生成 ===")
    print(f"{'='*60}")
    
    if 'A/C Name' in data.columns:
        indoor_temp_col = 'Indoor Temp.'
        ac_set_temp_col = 'A/C Set Temperature'
        
        # A/C Name別分析
        ac_analysis_results = analyze_by_ac_name(data, indoor_temp_col, ac_set_temp_col)
        
        if ac_analysis_results:
            print(f"{building_name}: 総 {len(ac_analysis_results)}個のA/C Nameに対する分析完了")
            
            # A/C ON状態サンプル数計算
            sample_counts_df = calculate_ac_on_sample_counts(data)
            
            # A/C Fan Speed頻度計算
            fan_speed_frequency_df = calculate_fan_speed_frequency(data)
            
            # Excelファイル生成
            success = update_excel_with_building_data(building_name, ac_analysis_results, sample_counts_df, fan_speed_frequency_df)
            
            if success:
                print(f"\n=== {building_name} 更新されたシート一覧（A/C ON状態のみ） ===")
                print("1. Indoortemp平均 - Indoor Temp月別平均（A/C ON状態のみ）")
                print("2. 設定温度_平均値 - A/C Set Temperature月別平均（A/C ON状態のみ）") 
                print("3. Indoortemp標準偏差 - Indoor Temp月別標準偏差（A/C ON状態のみ）")
                print("4. 設定温度_標準偏差 - A/C Set Temperature月別標準偏差（A/C ON状態のみ）")
                print("5. 室内機別_サンプル数 - A/C ON状態月別サンプル数")
                print("6. FanSpeed頻度 - A/C Fan Speed月別頻度")
            else:
                print(f"{building_name}: Excelファイル生成に失敗しました。")
        else:
            print(f"{building_name}: A/C Name別分析を実行できません。")
    else:
        print(f"{building_name}: A/C Name列がありません。")
        print(f"利用可能な列: {list(data.columns)}")

print(f"\n{'='*60}")
print("=== すべての建物分析完了 ===")
print(f"{'='*60}")



=== Clea 建物分析とExcelファイル生成 ===
総 26個のA/C Name発見

=== G-22 分析中 ===
  分析完了: 15個の月

=== G-21 分析中 ===
  分析完了: 15個の月

=== G-24 分析中 ===
  分析完了: 15個の月

=== G-23 分析中 ===
  分析完了: 7個の月

=== D-1南1 分析中 ===
  分析完了: 15個の月

=== D-3南2 分析中 ===
  分析完了: 15個の月

=== D-6北1 分析中 ===
  分析完了: 15個の月

=== D-5南1 分析中 ===
  分析完了: 15個の月

=== A-26 分析中 ===
  分析完了: 15個の月

=== E-17 分析中 ===
  分析完了: 15個の月

=== E-11南3 分析中 ===
  分析完了: 15個の月

=== E-9南1 分析中 ===
  分析完了: 15個の月

=== E-14北2 分析中 ===
  分析完了: 15個の月

=== E-13北1 分析中 ===
  分析完了: 15個の月

=== E-10南2 分析中 ===
  分析完了: 15個の月

=== F-18 分析中 ===
  分析完了: 15個の月

=== D-4北2 分析中 ===
  分析完了: 14個の月

=== E-15北3 分析中 ===
  分析完了: 15個の月

=== D-7南2 分析中 ===
  分析完了: 14個の月

=== D-8北2 分析中 ===
  分析完了: 13個の月

=== E-16北4 分析中 ===
  分析完了: 15個の月

=== A-25 分析中 ===
  分析完了: 15個の月

=== E-12南4 分析中 ===
  分析完了: 15個の月

=== D-2北1 分析中 ===
  分析完了: 12個の月

=== F-19 分析中 ===
  分析完了: 15個の月

=== F-20 分析中 ===
  分析完了: 15個の月
Clea: 総 26個のA/C Nameに対する分析完了
発見されたFan Speed値: ['Medium' 'Low' 'High' 'Top' 'Auto']
Fan Speed頻度計算完了

KeyboardInterrupt: 